## Get the combined Dataframes from Jan-17 to Jun-17 for Pickups originating at LGA = 138

In [1]:
from tqdm import tqdm
import glob
import pandas as pd

In [2]:
csvFiles = glob.glob("../raw_data/taxidata/*.csv")

In [3]:
combined_df = []

for file in tqdm(csvFiles):
    df = pd.read_csv(file, index_col = None, header = 0)
    df = df.loc[df["PULocationID"]==138, :] # 138 is for LaGuardia Airport
    df = df.reset_index()
    df = df.drop("index", axis=1)
    combined_df.append(df)

100%|██████████| 6/6 [03:32<00:00, 35.66s/it]


In [4]:
final_df = pd.concat(combined_df, ignore_index=True)

In [5]:
final_df.shape

(1471188, 17)

### Cleaning Taxi Data

In [6]:
final_df.shape

(1471188, 17)

In [7]:
final_df.columnsumns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount'],
      dtype='object')

#### Removing the columns which are not required for analysis

In [9]:
final_df.drop(["VendorID", "RatecodeID", "store_and_fwd_flag", "payment_type", "fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge"], axis=1, inplace=True)

In [10]:
final_df.shape

(1471188, 7)

#### Converting the pickup and dropoff timezones to localized America/New_York Datetimeindex

In [12]:
final_df.dtypes

tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
PULocationID               int64
DOLocationID               int64
total_amount             float64
dtype: object

In [13]:
final_df["tpep_pickup_datetime"] = pd.DatetimeIndex(final_df.tpep_pickup_datetime).tz_localize('America/New_York')
final_df["tpep_dropoff_datetime"] = pd.DatetimeIndex(final_df.tpep_dropoff_datetime).tz_localize('America/New_York')

In [14]:
final_df.dtypes

tpep_pickup_datetime     datetime64[ns, America/New_York]
tpep_dropoff_datetime    datetime64[ns, America/New_York]
passenger_count                                     int64
trip_distance                                     float64
PULocationID                                        int64
DOLocationID                                        int64
total_amount                                      float64
dtype: object

In [15]:
final_df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,total_amount
0,2017-01-01 00:00:05-05:00,2017-01-01 00:15:36-05:00,1,8.47,138,262,38.55
1,2017-01-01 00:00:12-05:00,2017-01-01 00:09:48-05:00,1,5.30,138,192,17.30
2,2017-01-01 00:00:17-05:00,2017-01-01 00:19:40-05:00,2,13.97,138,181,50.44
3,2017-01-01 00:00:21-05:00,2017-01-01 00:12:09-05:00,1,6.90,138,80,21.30
4,2017-01-01 00:00:33-05:00,2017-01-01 00:31:58-05:00,2,10.50,138,230,42.34


In [ ]:
final_df.to_csv('../clean_data/TaxiData_Jan17-Jun17.gz', compression='gzip', index=False)